<a href="https://colab.research.google.com/github/N40K0/VtuberDiscover/blob/main/VtuberDiscovery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img, load_img

# **Creating Functions**
- CreateData 
- trainModel

In [3]:
def createData(pathToDataset: str, imageSize: tuple, trainDataGenerator:ImageDataGenerator,
               valDataGenerator:ImageDataGenerator = ImageDataGenerator(rescale=1./255), 
               batchSize:int = 32, classMode = "sparse"):
    
    validationData = valDataGenerator.flow_from_directory(
        pathToDataset,
        target_size = imageSize,
        class_mode = classMode,
        seed = 123
    )

    trainingData = trainDataGenerator.flow_from_directory(
        pathToDataset,
        target_size = imageSize,
        class_mode = classMode,
        seed = 123
    )

    return trainingData, validationData

In [4]:
def trainingConfig(model ,trainingData, validationData, epochs, verbose, initialEpoch):
    checkPoint = keras.callbacks.ModelCheckpoint(
        "/content/CheckPoint", monitor='val_accuracy', save_best_only=True,
        mode='auto', save_freq='epoch', save_weights_only=True
    )

    return model.fit(
        trainingData,
        validation_data = validationData,
        epochs = epochs,
        initial_epoch = initialEpoch,
        callbacks = [checkPoint],
        verbose = verbose
    )

def trainModel(model:Model, trainingData, validationData, epochs = 11, verbose = 2, initialEpoch = 0):
    if tf.test.gpu_device_name() != '/device:GPU:0': 
        return trainingConfig(model, trainingData, validationData, epochs, verbose, initialEpoch)
    else:
        with tf.device("/device:GPU:0"):
            return trainingConfig(model, trainingData, validationData, epochs, verbose, initialEpoch)

# **Preparing data**

In [ ]:
!rm -rf /content/Vtubers
!cp /content/drive/MyDrive/Asset/Model/Datasets/VtuberDataset.tar.gz /content
!tar -xf /content/VtuberDataset.tar.gz

In [6]:
imageSize = (224, 224)
inputShape = (imageSize[0], imageSize[1], 3)

In [7]:
DataGenerator = ImageDataGenerator(
    rescale = 1./255,
    # width_shift_range= 0.2,
    # height_shift_range= 0.2,
    horizontal_flip = True,
    vertical_flip= True,
    zoom_range = 0.3,
    rotation_range = 20,
)

In [ ]:
trainingData, validationData = createData(
    trainDataGenerator = DataGenerator, 
    pathToDataset = "/content/Vtubers", 
    imageSize = imageSize,
    classMode = "sparse" 
)

Found 1446 images belonging to 5 classes.
Found 1446 images belonging to 5 classes.


# Creating Model


In [ ]:
model = keras.Sequential([
    layers.InputLayer(input_shape = inputShape),

    layers.Conv2D(128, (3,3), padding="valid", activation="relu"),
    layers.Dropout(0.2),
    layers.MaxPooling2D(pool_size = (2,2)),

    layers.Conv2D(256, (3,3), padding="valid", activation="relu"),
    layers.Dropout(0.2),
    layers.MaxPooling2D(pool_size = (2,2)),

    layers.Conv2D(512, (3,3), padding="valid", activation="relu"),
    layers.Dropout(0.2),
    layers.MaxPooling2D(pool_size = (2,2)),

    layers.Normalization(),
    layers.GlobalAveragePooling2D(),
    layers.Flatten(),
    layers.Dense(512, activation="relu"),
    layers.Dense(512, activation="relu"),
    layers.Dropout(0.8),
    layers.Dense(512, activation="relu"),
    layers.Dense(len(trainingData.class_indices), activation="softmax")
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 222, 222, 128)     3584      
                                                                 
 dropout_8 (Dropout)         (None, 222, 222, 128)     0         
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 111, 111, 128)    0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 109, 109, 256)     295168    
                                                                 
 dropout_9 (Dropout)         (None, 109, 109, 256)     0         
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 54, 54, 256)      0         
 2D)                                                  

In [ ]:
model.compile(
    optimizer = keras.optimizers.Adam(learning_rate=1e-4),
    loss = keras.losses.SparseCategoricalCrossentropy(),
    metrics = ["accuracy"]
)

# Train Model


In [ ]:
history = trainModel(model, trainingData, validationData, epochs=15, verbose=1, initialEpoch=0)    

Epoch 1/15
46/46 [==============================] - 38s 800ms/step - loss: 1.1037 - accuracy: 0.6100 - val_loss: 1.4937 - val_accuracy: 0.8181
Epoch 2/15
46/46 [==============================] - 37s 801ms/step - loss: 0.5104 - accuracy: 0.8250 - val_loss: 1.3689 - val_accuracy: 0.5519
Epoch 3/15
46/46 [==============================] - 37s 793ms/step - loss: 0.3053 - accuracy: 0.8928 - val_loss: 1.2977 - val_accuracy: 0.4564
Epoch 4/15
46/46 [==============================] - 36s 801ms/step - loss: 0.1699 - accuracy: 0.9502 - val_loss: 1.1123 - val_accuracy: 0.5463
Epoch 5/15
46/46 [==============================] - 37s 789ms/step - loss: 0.0693 - accuracy: 0.9820 - val_loss: 0.9517 - val_accuracy: 0.7040
Epoch 6/15
46/46 [==============================] - 37s 796ms/step - loss: 0.0596 - accuracy: 0.9841 - val_loss: 0.6469 - val_accuracy: 0.8976
Epoch 7/15
46/46 [==============================] - 37s 791ms/step - loss: 0.0540 - accuracy: 0.9786 - val_loss: 0.5587 - val_accuracy: 0.7607

# Testing


In [ ]:
!rm -rf ./Sample

In [ ]:
!rm -rf ./Sample/.ipynb_checkpoints/
import os

imgProcessing = ImageDataGenerator(
    rescale= 1./255,
    horizontal_flip = True,
    rotation_range = 20
)

# checkPoint = model.load_weights("/content/CheckPoint/variables/variables.index")
dir = "/content/Sample"
samples = os.listdir(dir)
samples.sort()

newLabels = {'Ethel_chamomile': 0, 'Kamiko_Kana': 1, "Kaneko_hato": 2, "Prune": 3, "Suzune_corryn": 4}
labelValue = list(newLabels.values())
labelName = list(newLabels.keys())

# print(newLabels)
for sample in samples:
    path = os.path.join(dir, sample)
    print(path)

    loadedImg = load_img(path, target_size = imageSize)
    imgBatch = tf.expand_dims(img_to_array(loadedImg), 0, sample)
    img = imgProcessing.flow(imgBatch)

    result = model.predict(img)
    prediction = tf.nn.sigmoid(result[0])
    score = np.argmax(prediction)

    predictionResult = labelValue.index(score)
    print(f"Predicion result {prediction[score]*100:0.2f}% indicate as {labelName[predictionResult]} [{score}]")
    print(f"with file name = {sample}\n")

/content/Sample/Kana1.jpg
Predicion result 73.11% indicate as Kamiko_Kana [1]
with file name = Kana1.jpg

/content/Sample/Kana3.jpg
Predicion result 73.10% indicate as Kamiko_Kana [1]
with file name = Kana3.jpg

/content/Sample/Kane1.png
Predicion result 73.11% indicate as Kaneko_hato [2]
with file name = Kane1.png

/content/Sample/Kane2.png
Predicion result 73.11% indicate as Kaneko_hato [2]
with file name = Kane2.png

/content/Sample/Prune1.png
Predicion result 72.50% indicate as Prune [3]
with file name = Prune1.png

/content/Sample/Prune2.png
Predicion result 66.40% indicate as Kamiko_Kana [1]
with file name = Prune2.png

/content/Sample/corryn1.png
Predicion result 72.59% indicate as Suzune_corryn [4]
with file name = corryn1.png

/content/Sample/corryn2.png
Predicion result 71.89% indicate as Suzune_corryn [4]
with file name = corryn2.png

/content/Sample/corryn3.png
Predicion result 72.50% indicate as Suzune_corryn [4]
with file name = corryn3.png

/content/Sample/ethel1.png
Pre

In [ ]:
model.save("./912022_v2.h5")

In [ ]:
!pip3 install -q tensorflowjs

In [ ]:
!mkdir ./tmp
!tensorflowjs_converter --input_format keras --output_format=tfjs_graph_model "/content/912022_v2.h5" "/content/tmp"
!!zip -r /content/discoverVtuberGraph-0.1.0.zip /content/tmp/